## Install Required Python Libraries

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## Create MCP server.py file

The output of this is a file server.py

In [42]:
%%writefile server.py
from fastmcp import FastMCP

# Create MCP server
mcp = FastMCP(host="0.0.0.0", stateless_http=True)

# Define a tool
@mcp.tool
def hello(name: str) -> str:
    """
    Say hello to someone
    """
    return f"Hello, {name}!"

# Run the server
if __name__ == "__main__":
    mcp.run(transport="streamable-http")

Overwriting server.py


## Configure AgentCore Runtime Deployment

The output of is a docker file

In [43]:
import bedrock_agentcore_starter_toolkit

agentcore_runtime = bedrock_agentcore_starter_toolkit.Runtime()

response = agentcore_runtime.configure(
    entrypoint="server.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=boto_session.region_name,
    protocol="MCP",
    agent_name="mcp_server_iam",
)

Entrypoint parsed: file=/home/sagemaker-user/MCP_AgentCore/server.py, bedrock_agentcore_name=server
Memory disabled - agent will be stateless
Configuring BedrockAgentCore agent: mcp_server_iam



💡 No container engine found (Docker/Finch/Podman not installed)
✓ Default deployment uses CodeBuild (no container engine needed), For local 
builds, install Docker, Finch, or Podman


Memory disabled
Network mode: PUBLIC



⚠️ Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore 
requires 'linux/arm64', so local builds won't work.
Please use default launch command which will do a remote cross-platform build 
using code build.For deployment other options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-cu
stom.html



Generated Dockerfile: Dockerfile
Generated .dockerignore: /home/sagemaker-user/MCP_AgentCore/.dockerignore
Keeping 'mcp_server_iam' as default agent
Bedrock AgentCore configured: /home/sagemaker-user/MCP_AgentCore/.bedrock_agentcore.yaml


## Launch AgentCore MCP Server

This will use code build to create an ECR image, which becomes invokable via. AgentCore runtime

In [44]:
launch_result = agentcore_runtime.launch()

🚀 Launching Bedrock AgentCore (cloud mode - RECOMMENDED)...
   • Deploy Python code directly to runtime
   • No Docker required (DEFAULT behavior)
   • Production-ready deployment

💡 Deployment options:
   • runtime.launch()                → Cloud (current)
   • runtime.launch(local=True)      → Local development
Memory disabled - skipping memory creation
Starting CodeBuild ARM64 deployment for agent 'mcp_server_iam' to account 430118851078 (us-east-1)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: mcp_server_iam
ECR repository available: 430118851078.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-mcp_server_iam
Getting or creating execution role for agent: mcp_server_iam
Using AWS region: us-east-1, account ID: 430118851078
Role name: AmazonBedrockAgentCoreSDKRuntime-us-east-1-22e9392f39


✅ Reusing existing ECR repository: 430118851078.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-mcp_server_iam


✅ Reusing existing execution role: arn:aws:iam::430118851078:role/AmazonBedrockAgentCoreSDKRuntime-us-east-1-22e9392f39
Execution role available: arn:aws:iam::430118851078:role/AmazonBedrockAgentCoreSDKRuntime-us-east-1-22e9392f39
Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: mcp_server_iam
Role name: AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-22e9392f39
Reusing existing CodeBuild execution role: arn:aws:iam::430118851078:role/AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-22e9392f39
Using dockerignore.template with 46 patterns for zip filtering
Uploaded source to S3: mcp_server_iam/source.zip
Updated CodeBuild project: bedrock-agentcore-mcp_server_iam-builder
Starting CodeBuild build (this may take several minutes)...
Starting CodeBuild monitoring...
🔄 QUEUED started (total: 0s)
✅ QUEUED completed in 1.0s
🔄 PROVISIONING started (total: 1s)
✅ PROVISIONING completed in 8.3s
🔄 DOWNLOAD_SOURCE started (total: 9s)
✅ DOWNLOAD_SOU

View the details of the launch

In [46]:
#print(launch_result)
print(f'Deployment mode: {launch_result.mode}')
print(f'Codebuild build run: {launch_result.codebuild_id}')
print('-' * 50)
print(f'Elastic Container Registry URI: {launch_result.ecr_uri}')
print('-' * 50)
print(f'Agent ID: {launch_result.agent_id}')
print(f'Agent ARN: {launch_result.agent_arn}')

Deployment mode: codebuild
Codebuild build run: bedrock-agentcore-mcp_server_iam-builder:d20159d8-e830-40a0-b186-2272e3914b78
--------------------------------------------------
Elastic Container Registry URI: 430118851078.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-mcp_server_iam
--------------------------------------------------
Agent ID: mcp_server_iam-5gwlsrFU6N
Agent ARN: arn:aws:bedrock-agentcore:us-east-1:430118851078:runtime/mcp_server_iam-5gwlsrFU6N


## Invoke the MCP server via. AgentCore Runtime Compute

In [92]:
%%writefile mcp_client_remote.py

import asyncio
import boto3
import mcp 
import streamable_http_sigv4

def create_streamable_http_transport_sigv4(mcp_url: str, service_name: str, region: str):
    session = boto3.Session()
    credentials = session.get_credentials()

    # Create and return the custom transport with SigV4 signing capability
    return streamable_http_sigv4.streamablehttp_client_with_sigv4(
        url=mcp_url,
        credentials=credentials,
        service=service_name,
        region=region,
    )

async def main():
    boto_session = boto3.session.Session()
    region = boto_session.region_name
    
    agent_arn = "arn:aws:bedrock-agentcore:us-east-1:430118851078:runtime/mcp_server_iam-5gwlsrFU6N"
    encoded_arn = agent_arn.replace(":", "%3A").replace("/", "%2F")
    
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"

    async with create_streamable_http_transport_sigv4(mcp_url=mcp_url, service_name="bedrock-agentcore", region=region) as (read_stream, write_stream, _,):
        async with mcp.ClientSession(read_stream, write_stream) as session:
            await session.initialize()
                    
            tool_result = await session.list_tools()
    
            print("Available MCP Tools:")
            for tool in tool_result.tools:
                print(f"Tool name: {tool.name}")
                print(f" Description: {tool.description}")

if __name__ == "__main__":
    asyncio.run(main())

Overwriting mcp_client_remote.py


In [93]:
!python mcp_client_remote.py

Available MCP Tools:
Tool name: hello
 Description: Say hello to someone
Session termination failed: 404
